In [27]:
from pyspark import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, substring
from pyspark.sql.types import IntegerType
import pyspark.sql.functions as f

spark = SparkSession.builder.master("local[1]") \
                    .appName('py_spark') \
                    .getOrCreate()


csvDF =spark.read.option("header",True).csv("/home/james/python_practice/archive/CAvideos.csv")

jsonDF =spark.read.option("multiLine", True).json("/home/james/python_practice/archive/CA_category_id.json").rdd.flatMap(lambda x:x["items"]).toDF()

joinDF=csvDF.join(jsonDF, csvDF['category_id'] == jsonDF['id'] )

joinDF =joinDF.withColumn("dislikes",  col('dislikes').cast(IntegerType()))
joinDF=joinDF.select(joinDF["dislikes"],joinDF['snippet.title'])
joinDF.show()
joinDF =joinDF.groupBy('title').agg(f.sum('dislikes').alias('dislikes')).sort(f.col('dislikes').desc()).head(10)
spark.createDataFrame(joinDF).show()
# joinDF.show(100)

+--------+----------------+
|dislikes|           title|
+--------+----------------+
|    1204|Film & Animation|
|     348|Film & Animation|
|    2520|Film & Animation|
|      84|Film & Animation|
|      27|Film & Animation|
|     831|Film & Animation|
|   60025|Film & Animation|
|    1985|Film & Animation|
|    3657|Film & Animation|
|    5508|Film & Animation|
|     648|Film & Animation|
|    8506|Film & Animation|
|     569|Film & Animation|
|     224|Film & Animation|
|    8927|Film & Animation|
|      60|Film & Animation|
|     543|Film & Animation|
|     956|Film & Animation|
|    1135|Film & Animation|
|   51293|Film & Animation|
+--------+----------------+
only showing top 20 rows



+--------------------+--------+
|               title|dislikes|
+--------------------+--------+
|       Entertainment|25791583|
|               Music|22098190|
|      People & Blogs| 9480796|
|              Comedy| 5372515|
|              Sports| 3286369|
|     News & Politics| 3264041|
|    Film & Animation| 3111259|
|              Gaming| 2446882|
|Science & Technology| 1791153|
|       Howto & Style| 1730520|
+--------------------+--------+

